# Benchmarks

A major priority of this package is to ensure the algorithms provided for fitting the SPE spectra are optimised to minimise execution time. Changes to the package should be assessed against this benchmarking notebook to ensure the changes have not resulted in a reduced efficiency.

In [1]:
from spefit.pdf import PDF, PMTSingleGaussian
from spefit.cost import Cost
from spefit.container import ChargeContainer
from spefit.fitter import CameraFitter, minimize_with_iminuit
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2e}'.format

/Users/Jason/Software/spefit/spefit/fitter.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


## 1. PDF Class Calling

In [2]:
x = np.linspace(-5, 10, 1000)
time_dict = {}
for pdf_class in PDF.__subclasses__():
    pdf = pdf_class(n_illuminations=3)
    parameters = np.array(list(pdf.initial.values()))  # Use the initial values set in the PDF
    time = %timeit -q -o pdf(x, parameters, 0)
    time_dict[pdf_class.__name__] = time.average

In [3]:
df = pd.DataFrame.from_dict(time_dict, orient="index")
df.rename({0: "Time (seconds)"}, axis='columns')

,Time (seconds)
PMTSingleGaussian,7.01e-05
SiPMGentile,1.07e-04
SiPMModifiedPoisson,1.13e-04


## 2. Cost Calling

In [4]:
x = np.linspace(-5, 10, 1000)
pdf = PMTSingleGaussian(n_illuminations=3)
parameters = np.array(list(pdf.initial.values()))  # Use the initial values set in the PDF
charges = []
for i in range(pdf.n_illuminations):
    y = pdf(x, parameters, i)  # Calculate PDF via the direct call to the PDF function
    c = np.random.choice(x, p=y / y.sum(), size=20000)  # Inverse transform sampling
    charges.append(ChargeContainer(c, n_bins=100, range_=(-3, 6)))

In [5]:
time_dict = {}
for cost_class in Cost.__subclasses__():    
    cost = cost_class(pdf, charges)
    time = %timeit -q -o cost(parameters)
    time_dict[cost_class.__name__] = time.average

In [6]:
df = pd.DataFrame.from_dict(time_dict, orient="index")
df.rename({0: "Time (seconds)"}, axis='columns')

,Time (seconds)
UnbinnedNLL,4.20e-03
BinnedNLL,5.81e-05
LeastSquares,4.33e-05


## 3. Cost Minimization

In [7]:
time_dict = {}
for cost_class in Cost.__subclasses__():    
    cost = cost_class(pdf, charges)
    time = %timeit -q -o minimize_with_iminuit(cost)
    time_dict[cost_class.__name__] = time.average

In [8]:
df = pd.DataFrame.from_dict(time_dict, orient="index")
df.rename({0: "Time (seconds)"}, axis='columns')

,Time (seconds)
UnbinnedNLL,5.50e-01
BinnedNLL,8.52e-03
LeastSquares,1.07e-02


## 4. Camera Fitting

In [9]:
pixel_charges = [c.values[:, None] * np.ones((1, 3000)) for c in charges]
fitter = CameraFitter(pdf, 100, (-3, 6), "BinnedNLL")

In [10]:
fitter.process(pixel_charges)

  0%|          | 0/3000 [00:00<?, ?it/s]

In [11]:
fitter.multiprocess(pixel_charges, n_processes=3)

Multiprocessing pixel SPE fit (n_processes = 3)


  0%|          | 0/3000 [00:00<?, ?it/s]